# Usage of the solver

In [13]:
# General setup
from numpy.random import randint, choice

explain some intro stuff here maybe

In [280]:
# idk if random generation is the right idea,
# maybe a more systematic approach would be better.
# just trying things out for now.
class Model():
    def __init__(self, worlds: list = None, agents: list = None,
                 relations=None, valuations=None):
        self.worlds = worlds
        self.agents = agents
        self.relations = relations
        self.valuations = valuations
        if self.valuations is None:
            self.create_valuations()
        if self.relations is None:
            self.create_relations()
        if self.agents is None:
            self.create_agents()
        if self.worlds is None:
            self.create_worlds()

    def create_worlds(self, max: int = 20):
        # generate a random number of worlds
        self.worlds = []
        for i in range(randint(1, max + 1)):
            self.worlds.append(f"w{i}")
        return self.worlds

    def create_agents(self, max: int = 26):
        # generate a random number of agents
        self.agents = []
        for i in range(randint(1, max + 1)):
            self.agents.append(chr(ord('A')+i))
        return self.agents

    def create_relations(self):
        # generate random relations
        if self.worlds is None:
            self.create_worlds()
        if self.agents is None:
            self.create_agents()
        # This should make S5 models, doesn't do that yet
        self.relations = {}
        for a in self.agents:
            self.relations[a] = {}
            for world in self.worlds:
                self.relations[a][world] = {w for w in self.worlds
                                            if w == world
                                            or randint(0, 10) < 5}
        # print(self.relations)
        return self.relations

    def create_valuations(self, max: int = 3):
        # generate random valuations
        if self.worlds is None:
            self.create_worlds()

        atoms = [chr(ord('p')+i) for i in range(randint(1, max + 1))]
        self.valuations = {}
        for atom in atoms:
            self.valuations[atom] = {w for w in self.worlds
                                     if randint(0, 10) < 5}
        # print(self.valuations)
        return self.valuations

In [281]:
def create_formula(model: Model, ldepth:int = 0, hdepth:int = 12) -> str:
    formula = ""

    for _ in range(randint(1,hdepth)):
        if randint(5) == 1:
            formula += '!'
        formula += "K" + choice(model.agents)
    if True: #randint(10) != 1:
        if randint(5) == 1:
            formula += '!'
        formula += choice(list(model.valuations.keys()))
    else: 
        formula += '(' + create_formula(model) + ")&(" + create_formula(model) + ')'

    return formula 

In [272]:
print(create_formula(model))

KAKAKAp


In [ ]:
class Solver:
    def __init__(self, formula, model: Model = None, verbose: bool = False):
        """
        Initialize the solver with a starting formula, set of worlds, relations and valuations

        To Do: Add support for NOT operator, and, or etc.

        :param formula: Description
        :param model:
        :param verbose:
        """
        self.start = formula
        self.level = 0
        self.f = formula
        self.w = model.worlds if model.worlds else model.create_worlds()
        self.r = model.relations if model.relations else model.create_relations()
        self.v = model.valuations if model.valuations else model.create_valuations()
        self.talks = verbose
        print(self.talks)

    def eval_formula(self, formula, world):
        """maybe hashmap this to be more efficient also it seems like its broken..."""
        if formula[0] in self.v:
            # print(world, formula[0], world in self.v.get(formula[0], set())) 
            return world in self.v.get(formula[0], set())
        if formula[0] == "K":
            agent, sub_formula = formula[1], formula[2:]
            return all(self.eval_formula(sub_formula, connected_world)
                       for connected_world in self.r[agent][world])
        if formula[0] == "!":
            print('not', formula[1:])
            return not(self.eval_formula(formula[1:], world))
        if formula[0] == "(":
            sub_forms = self.parse_brackets(formula)
            this_bracket = [sub_formula for i, sub_formula in sub_forms if i == 0]
            return all(self.eval_formula(sub_formula, world) 
                    for sub_formula in this_bracket)


    def parse_brackets(self, string):
        # Copied from Gareth Rees on stackOverflow
        stack = []
        for i, c in enumerate(string):
            if c == '(':
                stack.append(i)
            elif c == ')' and stack:
                start = stack.pop()
                yield (len(stack), string[start + 1: i])


    def satisfying_worlds(self):
        return {w for w in self.w if self.eval_formula(self.start, w)}

    def strip_outer_K(self):
        self.f = self.f[2:] if self.f[0] == "K" else self.f

    def analyze(self):
        answers = []

        while True:
            ws = self.satisfying_worlds()

            if self.talks:
                print("\n")
                print("Formula:", self.f)
                print("Possible worlds:", ws)
            

            if not ws:
                if self.talks:
                    print("→ inconsistent announcement")
                answers.append(None)
            elif len(ws) == 1:
                ans = next(iter(ws))
                if self.talks:
                    print("→ unique world:", ans)
                answers.append(ans)
            else:
                answer = []
                print(self.v)
                for atom in self.v:
                    truths = [w in self.v[atom] for w in ws]
                    if all(truths):
                        answer.append(atom)
                    elif not(any(truths)):
                        answer.append(f"!{atom}")
                # p_vals = {self.eval_formula(("p",), w) for w in ws}
                if self.talks:
                    print(f"My answer is → {', '.join(answer)}")
                answers.append(", ".join(answer))

            if self.f[0] != "K":
                break

            self.strip_outer_K()
            self.level += 1

        unique_answers = [a for a in answers if a is not None]
        # if (len(unique_answers) == len(set(unique_answers))
        #         and len(unique_answers) == len(answers)):
        if len(answers) != 0 and len(answers) == len(set(answers)):
            print("\nUnique different answer found for each K operator removed.")
            print("Formula:", self.start)
            print("worlds:", self.w)
            print("relations:", self.r)
            print("valuations:", self.v)
            return True
        else:
            # print("\t\tNo unique different answer found for each K operator removed.")
            print('.', end='')
            return False


To Do: 
- make sure model x formula combinations aren't repeated?
- generate better random search. this is a mess currently.

In [ ]:
for i in range(10):
    model = Model()
    formula = create_formula(model)
    if Solver(formula, model).analyze():
        print(model, formula)


In [ ]:
form = 'KB!q'
rel = {'A': {'w0': {'w0', 'w1'}, 'w1': {'w0', 'w1'}}, 'B':{'w0': {}, 'w1': {}}}
val = {'p': {'w0', 'w1'}, 'q': {'w0'}}
agents = ['A', 'B']
wor = ['w0', 'w1']

print(eval("(1+1) * 4"))

Solver(form, Model(wor, agents, rel, val), True).analyze()

8
True


Formula: KB!q
Possible worlds: {'w1', 'w0'}
{'p': {'w1', 'w0'}, 'q': {'w0'}}
My answer is → p


Formula: !q
Possible worlds: {'w1', 'w0'}
{'p': {'w1', 'w0'}, 'q': {'w0'}}
My answer is → p
.

False

In [173]:
import re

print(create_formula(model))
re.findall(r"\((.*?)\)", create_formula(model))

KA(KAp)&(KAp)


[]